In [2]:
import gradio as gr
from openai import OpenAI
import openai
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory 
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain


load_dotenv()
open_api_key = os.getenv("api_key")
os.environ['OPENAI_API_KEY'] = open_api_key

# 템플릿 설정 - 영어로 설정하는 것이 정확도 및 속도에서 더 좋은 성능을 발휘합니다.
system_ai = "You are an artificial intelligence assistant, and you should be able to respond to various topics such as reading PDFs, summarizing documents, managing schedules, providing weather information, searching for the shortest route, and conducting web searches. Please respond briefly in Korean."
system_ai2= "당신은 인공지능 비서야, pdf 읽기, 문서요약하기, 일정 관리, 날씨, 최단경로 검색, 웹 검색 등 다양한 내용에 답변할 수 있어야해"
system_setting = SystemMessagePromptTemplate.from_template(system_ai)
# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    system_setting,                                           # 역할부여
    MessagesPlaceholder(variable_name="HeyMate"),              # 메모리 저장소 설정. ConversationBufferMemory의 memory_key 와 동일하게 설정
    HumanMessagePromptTemplate.from_template("{master_user}"), # 사용자 메시지 injection
])
# 메모리 설정
memory = ConversationBufferWindowMemory(memory_key="HeyMate", 
                                  ai_prefix="AI 비서 HeyMate",
                                  human_prefix="사용자",
                                  return_messages=True,
                                  k=10)
# llm 모델 정의
chatgpt = ChatOpenAI(
    temperature=0.3,
    max_tokens=2048,
    model_name="gpt-3.5-turbo"
    ) 
# llmchain 정의
conversation = LLMChain(
    llm=chatgpt,   # LLM
    prompt=prompt, # Prompt
    verbose=True,  # True 로 설정시 로그 출력
    memory=memory  # 메모리
)

/home/yth1133/miniconda3/envs/heymate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
answer = conversation({"master_user": "안녕하세요"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an artificial intelligence assistant, and you should be able to respond to various topics such as reading PDFs, summarizing documents, managing schedules, providing weather information, searching for the shortest route, and conducting web searches. Please respond briefly in Korean.
Human: 안녕하세요

> Finished chain.


In [8]:
memory

ConversationBufferWindowMemory(return_messages=True, human_prefix='사용자', ai_prefix='AI 비서 HeyMate', memory_key='HeyMate', k=10)

In [12]:
memory.chat_memory.messages[0].content, memory.chat_memory.messages[1].content

('안녕하세요', '안녕하세요! 무엇을 도와드릴까요?')

In [13]:
history = [x.content for x in memory.chat_memory.messages]
history

['안녕하세요', '안녕하세요! 무엇을 도와드릴까요?']

In [18]:
history2 = list(zip(history[0::2], history[1::2]))

In [19]:
history2

[('안녕하세요', '안녕하세요! 무엇을 도와드릴까요?')]

In [9]:
from gtts import gTTS
import os

text = "안녕하세요, 반갑습니다."
tts = gTTS(text=text, lang='ko')
tts.save("output.mp3")

In [11]:
os.system("start output.mp3")

sh: 1: start: not found


32512